Get all Bathing Waters in this project from Wikidata and checks if they are connected to an OSM object using an [API](https://osm.wikidata.link/tagged)

* The project: European bathwaters [GITHUB](https://github.com/salgo60/EuropeanBathingWater/blob/main/README.md) / Wikidata
  * [Wikidata EBath](https://www.wikidata.org/wiki/Wikidata:WikiProject_European_Bath_Waters)
* this [Notebook](https://github.com/salgo60/EuropeanBathingWater/blob/main/Jupyter/OSM_Wikidata_EuropeanBathwater.ipynb)

* API [Wikidata to OpenStreetMap](https://osm.wikidata.link/tagged)
  * eg. [https://osm.wikidata.link/tagged/api/item/Q106708773](https://osm.wikidata.link/tagged/api/item/Q106708773)

* Another tool [osm.wikidata.link](https://osm.wikidata.link/search)
* OSM Sparql -> [users who has added most osmt:leisure "swimming_area"](https://tinyurl.com/ygj5strx) 

  
Status:  



| Date | Bathing waters | no WD - OSM | Lake | no WD - OSM |nature reserve | no WD - OSM  |
| ------------- |:-------------:|:-------------:|:-------------:|:-------------:|:-------------:| -----:|
| 20210614 | 3313 | 2163 | 1314 | 178 | 216 | 15
| 20210616 | 3333 | 2172 | 1324 | 179 | 216 | 15

TODO: 
* 

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-06-16 12:07:03.415758


In [2]:
import pandas as pd


In [3]:
#
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
 
# https://w.wiki/3UnN
queryBath = """SELECT (REPLACE(STR(?nodeBath), ".*Q", "Q") AS ?qbathid) ?nodeBathLabel ?bathingWaterIdentifier 
(REPLACE(STR(?nodeWater), ".*Q", "Q") AS ?qlakeid) ?nodeWaterLabel WHERE {
  VALUES ?nodeProjBath {wd:Q107096245 wd:Q106774536}
  ?nodeBath wdt:P6104 ?nodeProjBath.
  minus   { ?nodeBath wikibase:propertyType ?type} # just bathwater not properties
  OPTIONAL {?nodeBath wdt:P9616 ?bathingWaterIdentifier}
  OPTIONAL {
    ?nodeBath wdt:P206 ?nodeWater.
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY (?nodeBathLabel)"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDBath = get_sparql_dataframe(endpoint_url, queryBath)
WDBath["Source"] = "WD"     
WDBath.shape

(3333, 6)

In [4]:
WDBath.head()

,qbathid,nodeBathLabel,bathingWaterIdentifier,qlakeid,nodeWaterLabel,Source
0,Q107098707,Q107098707,DENI_PR_TK25_3124_01,None,None,WD
1,Q107152077,Q107152077,GRBW029116049101,None,None,WD
2,Q107152185,Q107152185,GRBW029116038101,None,None,WD
3,Q12304289,Q12304289,DKBW687,Q10726446,Ålbækbukten,WD
4,Q12316102,Q12316102,DKBW26,Q104662,Öresund,WD


In [5]:
import urllib3, json
from tqdm import tqdm
http = urllib3.PoolManager()

listBath = []
#for WD, row in WDBath.iterrows():
for WD, row in tqdm(WDBath.iterrows(), total=WDBath.shape[0]):
    #print(row["qbathid"] ) 
    url = "https://osm.wikidata.link/tagged/api/item/" + row["qbathid"] 
    
    new_item = dict()
    new_item['wikidata'] = row["qbathid"] 
#    new_item['coord'] = row["coord"] 
    try:
        r = http.request('GET', url) 
        data = json.loads(r.data.decode('utf-8'))
    except:
        print (r.status, url)
#    print (r.status)
    try:
        #print ("OSM ", data["osm"], "Type: ", type(data["osm"]))
        #print ("ID: ", data["osm"][0]["id"])
        osmid = data["osm"][0]["id"]        
    except:
        #print ("error")
        osmid =""
    new_item['osmid'] = osmid 
    listBath.append(new_item)
print (len(listBath))

100%|██████████| 3333/3333 [05:24<00:00, 10.27it/s]

3333


In [6]:
OSMtot = pd.DataFrame(listBath,
                  columns=['wikidata','coord','osmid'])
OSMtot.shape


(3333, 3)

In [7]:
pd.set_option('max_colwidth', 400)
OSMtot.head(10)

,wikidata,coord,osmid
0,Q107098707,NaN,
1,Q107152077,NaN,
2,Q107152185,NaN,
3,Q12304289,NaN,2214457035
4,Q12316102,NaN,
5,Q12330411,NaN,
6,Q12335052,NaN,
7,Q1566473,NaN,1446589
8,Q17489404,NaN,6956526
9,Q2933430,NaN,


In [8]:
#OSMempty = OSMtot.osmid.notnull()
OSMtot[(OSMtot['osmid']=="")].shape

(2185, 3)

In [9]:
OSMEmpty =OSMtot[(OSMtot['osmid']=="")]

In [10]:
OSMEmpty.shape

(2185, 3)

In [11]:
OSMEmpty.to_csv("WD - OSM Bathwaters missing.csv")

OSMEmpty.head()

,wikidata,coord,osmid
0,Q107098707,NaN,
1,Q107152077,NaN,
2,Q107152185,NaN,
4,Q12316102,NaN,
5,Q12330411,NaN,


### Check connected water

In [12]:
querySeaMinus_Kust = """SELECT (REPLACE(STR(?nodeBath), ".*Q", "Q") AS ?qbathid) ?nodeBathLabel ?bathingWaterIdentifier ?SJOID
(REPLACE(STR(?nodeWater), ".*Q", "Q") AS ?qlakeid) ?nodeWaterLabel ?coord WHERE {
  VALUES ?nodeProjBath {wd:Q107096245 wd:Q106774536}
  ?nodeBath wdt:P6104 ?nodeProjBath.
  minus   { ?nodeBath wikibase:propertyType ?type} # just bathwater not properties

  OPTIONAL {?nodeBath wdt:P9616 ?bathingWaterIdentifier}
  OPTIONAL { ?nodeBath wdt:P625 ?coord. }
   {
    ?nodeBath wdt:P206 ?nodeWater.
    OPTIONAL { ?nodeWater wdt:P761 ?SJOID. }
    minus {?nodeWater wdt:P31 wd:Q16615865}
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY (?nodeBathLabel)"""
WDSea = get_sparql_dataframe(endpoint_url, querySeaMinus_Kust)
WDSea["Source"] = "WD"     
WDSea.shape

(1324, 8)

In [13]:
WDSea.head()

,qbathid,nodeBathLabel,bathingWaterIdentifier,SJOID,qlakeid,nodeWaterLabel,coord,Source
0,Q12304289,Q12304289,DKBW687,None,Q10726446,Ålbækbukten,Point(10.4722 57.5114),WD
1,Q12316102,Q12316102,DKBW26,None,Q104662,Öresund,Point(12.5822 55.7329),WD
2,Q12330411,Q12330411,DKBW96,None,Q12326358,Masned Sund,Point(11.8725 55.0056),WD
3,Q12335052,Q12335052,DKBW219,None,Q3801,Limfjorden,Point(8.5925 56.6072),WD
4,Q17489404,Q17489404,DKBW908,None,Q1189156,Jammerbugten,Point(9.5813 57.2564),WD


In [14]:
listSea = []
#for WD, row in WDBath.iterrows():
for WD, row in tqdm(WDSea.iterrows(), total=WDSea.shape[0]):
    #print(row["qlakeid"] ) 
    url = "https://osm.wikidata.link/tagged/api/item/" + row["qlakeid"] 
    
    new_item = dict()
    new_item['wikidata'] = row["qlakeid"] 
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))
    try:
        #print ("ID: ", data["osm"][0]["id"])

        osmid = data["osm"][0]["id"] 
        
    except:
        #print ("error")
        osmid =""
    new_item['osmid'] = osmid 
    listSea.append(new_item)
print (len(listSea))
OSMSeatot = pd.DataFrame(listSea,
                  columns=['wikidata','osmid'])

100%|██████████| 1324/1324 [02:13<00:00,  9.92it/s]

1324


In [15]:
OSMSeatot.head()

,wikidata,osmid
0,Q10726446,4681567
1,Q104662,486348349
2,Q12326358,592044072
3,Q3801,8312667
4,Q1189156,4681568


In [16]:
OSMSeaEmpty=OSMSeatot[(OSMSeatot['osmid']=="")]
# OSMSeaEmpty.wikidata.unique()  
#pd.Series(OSMSeaEmpty.wikidata.unique().sort(),dtype=pd.StringDtype()).to_csv("WD - OSM Vatten saknas.csv")
pd.Series(OSMSeaEmpty.wikidata.unique()).to_csv("WD - OSM Watten missing.csv")

OSMSeaEmpty.shape


(225, 2)

# Check Nature reserve

In [17]:
# https://w.wiki/3N6L
queryNature = """SELECT (REPLACE(STR(?nodeBath), ".*Q", "Q") AS ?qbathid) ?nodeBathLabel ?bathingWaterIdentifier ?Naturreg
(REPLACE(STR(?naturreservat), ".*Q", "Q") AS ?naturreservatID)  WHERE {
  VALUES ?nodeProjBath {wd:Q107096245 wd:Q106774536}
  ?nodeBath wdt:P6104 ?nodeProjBath.
  OPTIONAL {?nodeBath wdt:P9616 ?bathingWaterIdentifier}

  ?nodeBath wdt:P3018 ?naturreservat.
  ?naturreservat wdt:P3613 ?Naturreg. # --> reservat
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
"""
WDNature = get_sparql_dataframe(endpoint_url, queryNature)
WDNature["Source"] = "WD"     
WDNature.shape

(216, 6)

In [18]:
WDNature.head()

,qbathid,nodeBathLabel,bathingWaterIdentifier,Naturreg,naturreservatID,Source
0,Q106707117,Larödbaden,SE0441283000000317,2045268,Q60732601,WD
1,Q106707119,Domsten,SE0441283000000319,2030989,Q29579930,WD
2,Q106707120,Utvälinges badplats,SE0441283000000320,2042687,Q10659042,WD
3,Q106707127,Gyllebosjöns badplats,SE0441291000000329,2014529,Q10511301,WD
4,Q106707141,badplats Ryssvägen,SE0441286000000348,2001805,Q27968545,WD


In [19]:
listNature = []
for WD, row in tqdm(WDNature.iterrows(), total=WDNature.shape[0]):
    #print(row["naturreservatID"] ) 
    url = "https://osm.wikidata.link/tagged/api/item/" + row["naturreservatID"] 
    
    new_item = dict()
    new_item['wikidata'] = row["naturreservatID"] 
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))
    try:
        #print ("ID: ", data["osm"][0]["id"])
        osmid = data["osm"][0]["id"] 
        
    except:
        #print ("error")
        osmid =""
    new_item['osmid'] = osmid 
    listNature.append(new_item)
print (len(listNature))
OSMNature = pd.DataFrame(listNature,
                  columns=['wikidata','osmid'])

100%|██████████| 216/216 [00:19<00:00, 11.03it/s]

216


In [20]:
OSMNature.head()

,wikidata,osmid
0,Q60732601,
1,Q29579930,913660002
2,Q10659042,12400357
3,Q10511301,1460544
4,Q27968545,102924516


In [21]:
OSMNatureEmpty=OSMNature[(OSMNature['osmid']=="")]
#OSMNatureEmpty.to_csv("WD - OSM Nature reserve missing.csv")
pd.Series(OSMNatureEmpty.wikidata.unique()).to_csv("WD - OSM Nature reserve missing.csv")

#OSMNatureEmpty

In [22]:
print("*", start_time.strftime("%Y%m%d"),"Bathing water", WDBath.shape[0], "not OSM connected",OSMEmpty.shape[0]) 
print(" * water connected", WDSea.shape[0], "not OSM connected",OSMSeaEmpty.shape[0])
print(" * naturreserve connected", OSMNature.shape[0], "not OSM connected",OSMNatureEmpty.shape[0])


* 20210616 Bathing water 3333 not OSM connected 2185
 * water connected 1324 not OSM connected 225
 * naturreserve connected 216 not OSM connected 27


Line for GITHUB summary

In [23]:
print("ej kopplade",OSMEmpty.shape[0],"vatten ej kopplade", OSMSeaEmpty.shape[0], \
      ", naturreservat ej kopplade",OSMNatureEmpty.shape[0])



ej kopplade 2185 vatten ej kopplade 225 , naturreservat ej kopplade 27


Generate Markdown table eg.
| 20210526     | 2802 | 2050 |1147 | 254 | 213| 84|


In [24]:
print("|",start_time.strftime("%Y%m%d"),"|", \
      WDBath.shape[0],"|",OSMEmpty.wikidata.nunique(),"|", \
      OSMSeatot.shape[0],"|", OSMSeaEmpty.wikidata.nunique(),"|", \
      OSMNature.shape[0],"|", OSMNatureEmpty.wikidata.nunique())


| 20210616 | 3333 | 2172 | 1324 | 179 | 216 | 15


In [25]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2021-06-16 12:15:21.961166
Time elapsed (hh:mm:ss.ms) 0:08:18.545846
